<a href="https://colab.research.google.com/github/VulgarBOMB/LAB2/blob/main/LAB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!mkdir task1
!mkdir task2
!mkdir task3

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, min, max, to_date, floor, coalesce, lit, months_between

spark = SparkSession.builder.master("local[*]") \
                    .appName('sparkProject') \
                    .getOrCreate()

jobsDF = spark.read.parquet("jobs.parquet")
employeesDF = spark.read.parquet("employees.parquet")
departmentsDF = spark.read.parquet("department.parquet")
locationsDF = spark.read.parquet("locations.parquet")
regionsDF = spark.read.parquet("regions.parquet")
countriesDF = spark.read.parquet("countries.parquet")
job_historyDF = spark.read.parquet("job_history.parquet")

# Проверка данных в паркетниках
Реализация проверок на типы данных, дубликаты и Null-значения

In [ ]:
def checkDataTypes(jobsDF, locationsDF, departmentsDF, employeesDF):
  if dict(locationsDF.dtypes)['location_id'] != 'int':
    print('Типы данных в LOCATIONS некорректные')
    return False
  elif dict(departmentsDF.dtypes)['department_id'] != 'int':
    print('Типы данных в DEPARTMENTS некорректные')
    return False
  elif dict(employeesDF.dtypes)['salary'] != 'float'\
    or dict(employeesDF.dtypes)['department_id'] != 'int':
    print('Типы данных в EMPLOYEES некорректные')
    return False
  elif dict(jobsDF.dtypes)['min_salary'] != 'float':
    print('Типы данных в JOBS некорректные')
    return False
  else:
    print('Все типы данных корректные')
    return True

def checkWholeNullItem(jobsDF, locationsDF, departmentsDF, employeesDF):
  if locationsDF.filter(
      col('location_id').isNull() |
      col('location_id').isNull()
  ).count() > 0:
    print('JOBS имеет NULL-значения:')
    locationsDF.filter((col('location_id').isNull()) |
                      (col('state_province').isNull())).show()
    return False
  elif departmentsDF.filter(
      col('department_id').isNull() |
      col('department_name').isNull()
  ).count() > 0:
    print('DEPARTMENTS имеет NULL-значения:')
    departmentsDF.filter((col('department_id').isNull()) |
                         (col('department_name').isNull())).show()
    return False
  elif employeesDF.filter(
      col('first_name').isNull() |
      col('last_name').isNull() |
      col('job_id').isNull() |
      col('salary').isNull() |
      col('commission_pct').isNull() |
      col('hire_date').isNull() |
      col('department_id').isNull()
  ).count() > 0:
    print('EMPLOYEES имеет NULL-значения:')
    employeesDF.filter((col('first_name').isNull()) |
                       (col('last_name').isNull()) |
                       (col('job_id').isNull()) |
                       (col('salary').isNull()) |
                       (col('commission_pct').isNull()) |
                       (col('hire_date').isNull()) |
                       (col('department_id').isNull())).show(10000)
    return False
  elif jobsDF.filter(
      col('job_id').isNull() |
      col('job_title').isNull() |
      col('min_salary').isNull()
  ).count() > 0:
    print('JOBS имеет NULL-значения:')
    jobsDF.filter((col('job_id').isNull()) |
                  (col('job_title').isNull()) |
                  (col('min_salary').isNull())).show()
    return False
  else:
    print('Нет NULL-значений в таблицах')
    return True

def checkDuplicateRows(jobsDF, locationsDF, departmentsDF, employeesDF):
  department_counts = departmentsDF\
    .groupBy("department_name")\
    .agg(count("*")\
    .alias("count"))\
    .filter(col("count") > 1)\
    .count()

  location_counts = locationsDF\
    .groupBy("location_id", "state_province")\
    .agg(count("*")\
    .alias("count"))\
    .filter(col("count") > 1)\
    .count()

  employee_counts = employeesDF\
    .groupBy("first_name", "last_name", "job_id", "salary", "department_id")\
    .agg(count("*")\
    .alias("count"))\
    .filter(col("count") > 1)\
    .count()

  job_counts = jobsDF\
    .groupBy("job_title")\
    .agg(count("*")\
    .alias("count"))\
    .filter(col("count") > 1)\
    .count()

  if (department_counts + location_counts + employee_counts + job_counts) != 0:
    print('В одной из таблиц найдены дубликаты')
    return False
  else:
    print('Дубликаты не были найдены')
    return True

# Задание 1

In [ ]:
departmentsDF.createOrReplaceTempView('departments')
employeesDF.createOrReplaceTempView('employees')
jobsDF.createOrReplaceTempView('jobs')

# spark.sql("""
# SELECT d.department_name,
#       e.employee_id,
#       e.first_name,
#       e.last_name,
#       e.salary,
#       j.max_salary,
#       (SELECT AVG(salary) FROM employees WHERE department_id = d.department_id) AS avg_salary
# FROM employees e
# LEFT JOIN departments d ON d.department_id=e.department_id
# LEFT JOIN jobs j ON e.job_id = j.job_id
# WHERE e.salary > (
#   SELECT AVG(salary)
#   FROM employees
#   WHERE department_id = d.department_id
# )
# ORDER BY d.department_name
# """).show()

if (checkDataTypes(jobsDF, locationsDF, departmentsDF, employeesDF) &
checkDuplicateRows(jobsDF, locationsDF, departmentsDF, employeesDF)):

  checkWholeNullItem(jobsDF, locationsDF, departmentsDF, employeesDF)

  joiningDF = employeesDF.join(departmentsDF, departmentsDF.department_id == employeesDF.department_id, "left").\
                          join(jobsDF, jobsDF.job_id == employeesDF.job_id, "left").\
                          select(departmentsDF.department_name, \
                                 employeesDF.employee_id, \
                                 employeesDF.first_name, \
                                 employeesDF.last_name, \
                                 employeesDF.salary, \
                                 jobsDF.max_salary, \
                                 employeesDF.department_id)
  avg_salaryDF = employeesDF.groupBy("department_id").\
                             agg(avg("salary").alias("avg_salary"))
  resultDF = joiningDF.join(avg_salaryDF, 'department_id', "left").\
                       drop('department_id').\
                       filter(joiningDF.salary > avg_salaryDF.avg_salary).\
                       orderBy(joiningDF.department_name)

  resultDF.write.mode('overwrite').parquet("/content/task1")
  resultDF.show()

else:
  print('Не пройдено одно из условий выше ↑')

# Задание 2

In [ ]:
employeesDF.createOrReplaceTempView("employees")

if (checkDataTypes(jobsDF, locationsDF, departmentsDF, employeesDF) &
checkDuplicateRows(jobsDF, locationsDF, departmentsDF, employeesDF)):

  checkWholeNullItem(jobsDF, locationsDF, departmentsDF, employeesDF)

  # Вариант с использованием DataFrame API
  resultDF = employeesDF.groupBy(employeesDF.job_id).\
             agg(avg(employeesDF.salary))
  resultDF.show()

  # # Вариант с использованием нативного SQL
  # resultDF = spark.sql("""
  # SELECT job_id, AVG(salary) FROM employees GROUP BY job_id;
  # """)

  # Преобразование в dictionary
  pair_rdd = resultDF.rdd.map(lambda x: (x[0], x[1]))
  # print(resultDF.rdd.collect())
  dictionary = pair_rdd.collectAsMap()
  print(dictionary)

  resultDF.write.mode('overwrite').parquet("/content/task2")

else:
  print('Не пройдено одно из условий выше ↑')

# Задание 3

In [ ]:
employeesDF = employeesDF.withColumn('hire_date', to_date('hire_date', 'dd.mm.yyyy'))
employeesDF.createOrReplaceTempView('employees')
departmentsDF.createOrReplaceTempView('departments')
locationsDF.createOrReplaceTempView('locations')

# spark.sql("""
# SELECT l.state_province,
#        e.employee_id,
#        e.hire_date,
#        floor(months_between('2017-10-01', e.hire_date)) as months,
#        e.salary,
#        COALESCE(e.commission_pct, 0) as commission_pct,
#        e.salary * months as earned_money_before_vat,
#        earned_money_before_vat - COALESCE((e.commission_pct * earned_money_before_vat), 0) as earned_money_after_vat
# FROM employees e
# LEFT JOIN departments d ON e.department_id = d.department_id
# LEFT JOIN locations l ON d.location_id = l.location_id
# WHERE e.hire_date <= '2017-10-01' AND e.salary*floor(months_between('2017-10-01', e.hire_date)) = (
#   SELECT max(e.salary * floor(months_between('2017-10-01', e.hire_date)))
#   FROM employees e
#   WHERE e.hire_date <= '2017-10-01'
# )
# UNION ALL
# SELECT l.state_province,
#        e.employee_id,
#        e.hire_date,
#        floor(months_between('2017-10-01', e.hire_date)) as months,
#        e.salary,
#        COALESCE(e.commission_pct, 0) as commission_pct,
#        e.salary * months as earned_money_before_vat,
#        earned_money_before_vat - COALESCE((e.commission_pct * earned_money_before_vat), 0) as earned_money_after_vat
# FROM employees e
# LEFT JOIN departments d ON e.department_id = d.department_id
# LEFT JOIN locations l ON d.location_id = l.location_id
# WHERE e.hire_date <= '2017-10-01' AND e.salary*floor(months_between('2017-10-01', e.hire_date)) = (
#   SELECT min(e.salary * floor(months_between('2017-10-01', e.hire_date)))
#   FROM employees e
#   WHERE e.hire_date <= '2017-10-01'
# );
# """).show(truncate=False)

# spark.sql("""
#  WITH salary_extremes AS (
#    SELECT
#      MAX(e.salary * floor(months_between('2017-10-01', e.hire_date))) as max_salary,
#     MIN(e.salary * floor(months_between('2017-10-01', e.hire_date))) as min_salary
#    FROM employees e
#    WHERE e.hire_date <= '2017-10-01'
#  )

#  SELECT l.state_province,
#        e.employee_id,
#        e.hire_date,
#        floor(months_between('2017-10-01', e.hire_date)) as months,
#        e.salary,
#        COALESCE(e.commission_pct, 0) as commission_pct,
#        e.salary * months as earned_money_before_vat,
#        earned_money_before_vat - COALESCE((e.commission_pct * earned_money_before_vat), 0) as earned_money_after_vat
#  FROM employees e
#  LEFT JOIN departments d ON e.department_id = d.department_id
#  LEFT JOIN locations l ON d.location_id = l.location_id,
#     salary_extremes se
#  WHERE e.hire_date <= '2017-10-01'
#  AND (e.salary*floor(months_between('2017-10-01', e.hire_date)) = se.max_salary
#  OR e.salary*floor(months_between('2017-10-01', e.hire_date)) = se.min_salary);
#  """).show(truncate=False)


if (checkDataTypes(jobsDF, locationsDF, departmentsDF, employeesDF) &
  checkDuplicateRows(jobsDF, locationsDF, departmentsDF, employeesDF)):

  checkWholeNullItem(jobsDF, locationsDF, departmentsDF, employeesDF)

  # Соединение dataframe
  df = employeesDF.alias('e')\
      .join(departmentsDF.alias('d'), col('e.department_id') == col('d.department_id'), 'left')\
      .join(locationsDF.alias('l'), col('d.location_id') == col('l.location_id'), 'left')

  # Создание калькулируемых столбцов
  df = df.withColumn('months', floor(months_between(lit('2017-10-01'), col('e.hire_date'))))\
      .withColumn('commission_pct', coalesce(col('e.commission_pct'), lit(0)))\
      .withColumn('earned_money_before_vat', col('e.salary') * col('months'))\
      .withColumn('earned_money_after_vat', col('earned_money_before_vat') - (col('commission_pct') * col('earned_money_before_vat')))\
      .select(col('l.state_province'),\
              col('e.employee_id'),\
              col('e.hire_date'),\
              col('months'),\
              col('e.salary'),\
              col('commission_pct'),\
              col('earned_money_before_vat'),\
              col('earned_money_after_vat'))

  # Получение мин и макс ЗП
  max_salary = df.filter(df.hire_date <= '2017-10-01')\
      .agg(max(df.salary * df.months)).collect()[0][0]
  print(df.collect()[0].__class__)
  min_salary = df.filter(df.hire_date <= '2017-10-01')\
      .agg(min(df.salary * df.months)).first()[0]

  # Конечный фильтр
  df = df.filter((df.hire_date <= '2017-10-01') & ((df.salary * df.months) == max_salary) | ((df.salary * df.months) == min_salary))
  df.show()
  df.write.mode('overwrite').parquet("/content/task3")

else:
  print('Не пройдено одно из условий выше ↑')